 # Low Contrast Detectability Evaluations with Pediatric-Sized QA Phantoms
 Brandon J. Nelson
 2023-02-03
 # Background
 This script generates plots from the `LCD_results.csv` file produced by `main_lcd_catphanSim.m` to visualize
 the relationships between phantom size, reconstruction method, lesion size, dose level on low contrast
 detectability in terms of area under the roc curve (AUC) and detectability signal to noise (SNR) which are
 outputs from the model observers available here <https://github.com/DIDSR/LCD_CT>
 ## Looking at the results from `main_lcd_catphanSim.m`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
lcd_data = pd.read_csv('/home/brandon.nelson/Dev/DLIR_Ped_Generalizability/geometric_phantom_studies/results/LCD/LCD_results.csv')

lcd_data.replace('dl_REDCNN', 'dlir', inplace=True)
lcd_data.rename(columns={'patient_diameter_mm': 'phantom diameter [mm]', 'dose_level_pct': 'dose [%]'}, inplace=True)
lcd_data = lcd_data[lcd_data['phantom diameter [mm]'] != 200] #ref has large fov
print(f"{len(lcd_data['observer'].unique())} observers X {len(lcd_data['phantom diameter [mm]'].unique())} phantom diameters\
      X {len(lcd_data['insert_HU'].unique())} lesion inserts per image X {len(lcd_data['recon'].unique())} recon types\
      X {len(lcd_data['dose [%]'].unique())} dose levels X {len(lcd_data['reader'].unique())} readers = {len(lcd_data)} LCD measurements")
lcd_data.head()

: 

In [ ]:
lcd_data.tail() 

: 

In [ ]:
insert_HU_size = {14 : '3 mm', 7: '5 mm', 5: '7 mm', 3: '10 mm'}
observers = lcd_data['observer'].unique()
observers

: 

 ## Getting the Mean and Standard Deviation
 Use the dataframe `groupby` method to group the data by the following groups (all groups except reader number), and then take the mean
 and standard deviation across `reader`. Leave `dose [%]` as last `groupby` item to be able plot against it later

In [ ]:
grouped = lcd_data.groupby(["phantom diameter [mm]","recon", "insert_HU", "observer", "dose [%]"])

lcd_mean = grouped.mean()
lcd_std = grouped.std()
lcd_mean

: 

In [ ]:
lcd_mean['auc'][112, 'dlir', 3, 'NPW 2D']

: 

 # AUC and SNR vs dose

 ## First build up our plotting routines

In [ ]:
from plot_LCD import LCD_Plotter

: 

## AUC and SNR vs dose for individual inserts
Can adjust the list of observers to update the plot

In [ ]:
plotter = LCD_Plotter(lcd_data)

: 

In [ ]:
plotter.recons = ['fbp' , 'dlir' , [ 'dlir', 'fbp']]

: 

In [ ]:
plotter.dose_levels = [100, 25]

: 

In [ ]:
plotter.insert_HUs = 7

: 

In [ ]:
plotter.observers = ['Laguerre-Gauss CHO 2D', 'NPW 2D']

: 

In [ ]:
plotter.plot?

: 

In [ ]:
plotter.plot(x='diameter', restype='snr', transpose=False, recon_cmp_method='div')

: 

Try and recreate each plot below using the Plotter class

In [ ]:
plotter = LCD_Plotter(lcd_data)
plotter.recons = 'fbp'
plotter.phantom_diameters = 292
plotter.insert_HUs = 14
plotter.plot(x='dose', restype='auc')

: 

Can also change the recon type to DLIR

In [ ]:
plotter.recons = 'dlir'
plotter.plot(x='dose', restype='auc')

: 

In [ ]:
plotter.insert_HUs = [14, 7, 5, 3]
plotter.plot(x='dose', restype='auc')

: 

entering 2 recon types in as a list compares them (takes the difference by default, see `recon_cmp_method`) 
if `recon_cmp_method = 'diff'` then `recon1 - recon2` if `recons = [[recon1, recon2]]` 

In [ ]:
plotter.recons = [['dlir', 'fbp']]
plotter.plot(x='dose', restype='snr')

: 

Can also specify which lesion inserts you want to view based on HU value(s), can enter single value or a list

In [ ]:
lcd_data['insert_HU'].unique()

: 

In [ ]:
plotter.insert_HUs = [5, 14]
plotter.plot(x='dose', restype='snr')

: 

 ### Let's look at everything (warning a bit overwhelming) AUC
 these could be included in the paper appendix, but for the main figures we'll want to distill this down to
 the main effects (described below)

In [ ]:
plotter.phantom_diameters

: 

In [ ]:
plotter = LCD_Plotter(lcd_data)
plotter.recons = ['fbp', 'dlir', ['dlir', 'fbp']]
plotter.phantom_diameters = sorted(plotter.phantom_diameters, reverse=True)
plotter.plot(x='dose', restype='auc')

: 

 ## First look at everything (warning a bit overwhelming) SNR
 Same as above, these could be included in the paper appendix, but for the main figures we'll want to distill this down to
 the main effects (described below)

In [ ]:
plotter.plot(x='dose', restype='snr')

: 

 ## Let's now break this down into smaller chunks to better understand the relationships between variables
 ### Starting with insert size and HU
 Let's first see if there's any noticeable difference in detectability based on insert size and contrast

 ### all 4 inserts auc vs dose (no diffs)
 this shows that there's not much difference between inserts

In [ ]:
plotter.phantom_diameters = [292, 185, 112]
plotter.recons = ['fbp', 'dlir']
plotter.plot(x='dose', restype='auc')

: 

### all 4 inserts snr vs dose (no diffs)
 this shows that there's not much difference between inserts

In [ ]:
plotter.plot(x='dose', restype='snr')

: 

 ## Now just 1 insert but show diffs auc

In [ ]:
plotter.insert_HUs = 14
plotter.recons = ['fbp', 'dlir', ['dlir', 'fbp']]
plotter.plot(x='dose', restype='auc')

: 

 ## Now just 1 insert but show diffs snr

In [ ]:
plotter.plot(x='dose', restype='snr')

: 

 # diff auc vs diam

  dlir - fbp diff auc

In [ ]:
plotter.reset()
plotter.dose_levels = 100
plotter.recons = 'fbp'
plotter.plot(x='diameter', restype='auc')

: 

In [ ]:
plotter.dose_levels = [100, 10]
plotter.recons = ['fbp', 'dlir', ['dlir', 'fbp']]
plotter.plot(restype='auc', x='diameter')

: 

In [ ]:
plotter.insert_HUs = 14
plotter.plot(restype='auc', x='diameter')

: 

 note above that sometimes NPWE can exceed NPW in \Delta AUC because the AUC is saturated, this cross-over doesn't occur in SNR
 since we showed this earlier we probs only need to show SNR moving forward in the paper if we show the auc saturation once

 dlir - fbp diff snr

In [ ]:
plotter.plot(restype='snr', x='diameter')

: 

 ## SNR ratios

In [ ]:
plotter.plot(restype='snr', x='diameter', recon_cmp_method='div')

: 

 the ratio becomes too noisy due to NPWE 2D (eye filter) so remove it from the list of observers

In [ ]:
plotter.observers = ['Laguerre-Gauss CHO 2D', 'NPW 2D']
plotter.dose_levels = [100, 55, 10]
plotter.plot(restype='snr', x='diameter', recon_cmp_method='div')

: 

In [ ]:
plotter.dose_levels = [100, 25]
plotter.plot(restype='snr', x='diameter', recon_cmp_method='div')

: 

In [ ]:
plotter.dose_levels = [100, 25]
plotter.insert_HUs = [14, 7, 5, 3]
plotter.recons = [['dlir', 'fbp']]
plotter.plot(restype='snr', x='diameter', recon_cmp_method='div', transpose = True)

: 

In [ ]:
plotter.insert_HUs = 7
plotter.recons = [['dlir', 'fbp']]
fig_dict = plotter.plot(restype='snr', x='diameter', recon_cmp_method='div', transpose = True)

: 

In [ ]:
ylim = (0.25, 6.75)
fig_dict = plotter.plot(restype='snr', x='diameter', recon_cmp_method='div', transpose = True)
fig_dict['fig0'][1][0].set_ylim(ylim)
fig_dict['fig0'][1][0].annotate("$d_{SNR}$ ratio = $\frac{DLIR d_{SNR}}{FBP d_{SNR}}$}",
                                xy=(0.7, 0.5))
fig_dict['fig1'][1][0].set_ylim(ylim)
# plt.savefig("SNR_ratio_v_diameter.png", dpi=600)
# fig_dict['fig0'][1][0].get_legend()

: 

In [ ]:
plotter = LCD_Plotter(lcd_data)

plotter.insert_HUs = 7
plotter.dose_levels = [25]
plotter.recons = ['fbp', 'dlir', ['dlir', 'fbp']]
plotter.observers = ['Laguerre-Gauss CHO 2D', 'NPW 2D']
# ylim = (0.25, 6.75)
fig_dict = plotter.plot(restype='auc', x='diameter', recon_cmp_method='diff', transpose = False)

: 

In [ ]:
plotter = LCD_Plotter(lcd_data)

plotter.insert_HUs = [3, 5, 7, 14]
plotter.dose_levels = [25]
plotter.recons = [['dlir', 'fbp']]
plotter.observers = ['Laguerre-Gauss CHO 2D', 'NPW 2D']
# ylim = (0.25, 6.75)
fig_dict = plotter.plot(restype='auc', x='diameter', recon_cmp_method='diff', transpose = False)

: 

In [ ]:
plotter = LCD_Plotter(lcd_data)

plotter.insert_HUs = [3, 5, 7, 14]
plotter.dose_levels = [100, 55, 25]
plotter.recons = [['dlir', 'fbp']]
plotter.observers = ['NPW 2D', 'NPWE 2D']
# ylim = (0.25, 6.75)
fig_dict = plotter.plot(restype='auc', x='diameter', recon_cmp_method='diff', transpose = False)

: 

In [ ]:
sf = fig_dict['fig0'][0]

: 

In [ ]:
sf.suptitle

: 

In [ ]:
  plt.style.use('seaborn-v0_8-deep')
    results_csv = results_csv or '/home/brandon.nelson/Dev/DLIR_Ped_Generalizability/geometric_phantom_studies/results/LCD/LCD_results.csv'
    results_csv = Path(results_csv)
    outputdir = outputdir or results_csv.parent
    outputdir = Path(outputdir)
    lcd_data = pd.read_csv(results_csv)

    lcd_data.replace({'dl_REDCNN': 'dlir',
                      'NPW 2D': 'NPW', 
                      'Laguerre-Gauss CHO 2D': 'Laguerre-Gauss CHO'}, inplace=True)
    lcd_data.rename(columns={'patient_diameter_mm': 'phantom diameter [mm]', 'dose_level_pct': 'dose [%]'}, inplace=True)
    lcd_data = lcd_data[lcd_data['phantom diameter [mm]'] != 200] #ref has large fov
    plotter = LCD_Plotter(lcd_data)

    plotter.insert_HUs = 7
    plotter.dose_levels = [25]
    plotter.recons = [['dlir', 'fbp']]
    plotter.observers = ['Laguerre-Gauss CHO', 'NPW'] # 'Laguerre-Gauss CHO 2D', 'NPW 2D'
    fig_dict = plotter.plot(restype=restype, x='diameter', recon_cmp_method=comparator, transpose=False)
    fig_dict['fig0'][1][0].set_ylim((-0.1, 0.4))
    fig_dict['fig0'][1][0].set_xlim((105, 308))
    fig_dict['fig0'][0].suptitle('')
    fig_dict['fig0'][1][0].set_title('')

    ages = [1, 5, 10, 15, 18]
    axs = [fig_dict['fig0'][1][0]]
    age_yloc = -0.075
    for ax in axs:
        ax.annotate('Age groups with\ncorresponding mean\nabdomen diameter', xy=(170, -0.08), xytext=(110, 0.2), arrowprops=dict(facecolor='black', shrink=0.2, alpha=0.25), fontsize=10)
        for a in ages:
            eff_diam = age_to_eff_diameter(a)*10
            ax.annotate(f'{a}yrs', xy=(eff_diam, age_yloc), xycoords='data', xytext=(eff_diam, age_yloc), ha='center', textcoords='data')

    f = plt.gcf()
    f.set_figheight(3)
    f.set_figwidth(3.2)
    output_fname = outputdir/f"{restype.upper()}_{comparator}_v_diameter.png"
    plt.savefig(output_fname, dpi=600, bbox_inches="tight")
    print(f'file saved: {output_fname}')

: 